In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import requests
import os
import warnings
from datetime import datetime, timedelta
from collections import defaultdict
import glob
from bs4 import BeautifulSoup
import networkx as nx
import unicodedata
from unidecode import unidecode

In [2]:
df_all = pd.read_csv("all_list.csv")

In [3]:
df_all['Player'] = df_all['Player'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_all['Club'] = df_all['Club'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

# from the colum "Player" remove str " (captain)" and " (vice-captain)"
df_all['Player'] = df_all['Player'].str.replace(' \(captain\)', '')


/var/folders/j1/k5mr2n390nbcsvcdpxrf9l8w0000gn/T/ipykernel_10933/2548669962.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  df_all['Player'] = df_all['Player'].str.replace(' \(captain\)', '')


In [4]:
# check players from spain
#df_all[df_all['Country'].str.contains('Denmark')]

In [5]:
# load data from FIFA23_official_data.csv
df_fifa = pd.read_csv("Fifa 23 Players Data.csv")
df_fifa['Full Name'] = df_fifa['Full Name'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
df_fifa['Known As'] = df_fifa['Known As'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')


# Check where 'Full Name' contains 'Hernandez'
#df_fifa[df_fifa['Club Name'].str.contains('Real Madrid')].head(50)


In [6]:
# Check where 'Nationality' contains 'Denmark'
#df_fifa[df_fifa['Club Name'].str.contains('Valencia')].head(50)

In [7]:
df_all['Player'].isin(df_fifa['Full Name']).value_counts()

#check 



True     578
False    252
Name: Player, dtype: int64

In [8]:
#check the false values
df_all[~df_all['Player'].isin(df_fifa['Full Name'])]

,Pos.,Player,Caps,Goals,Club,Country,team_id,Leauge_country
0,GK,Hernan Galindez,12,0,Aucas,Ecuador,1156.0,Equador
1,DF,Felix Torres,17,2,Santos Laguna,Ecuador,2285.0,mexico
3,DF,Robert Arboleda,33,2,Sao Paulo,Ecuador,NaN,NaN
8,MF,Ayrton Preciado,27,3,Santos Laguna,Ecuador,2285.0,mexico
9,MF,Romario Ibarra,25,3,Pachuca,Ecuador,2292.0,mexico
...,...,...,...,...,...,...,...,...
805,DF,Nestor Araujo,63,3,America,Mexico,NaN,NaN
814,FW,Rogelio Funes Mori,16,6,Monterrey,Mexico,2282.0,mexico
827,MF,Luis Chavez,9,0,Pachuca,Mexico,2292.0,mexico
828,FW,Roberto Alvarado,31,4,Guadalajara,Mexico,NaN,NaN


In [9]:
# combine the two dataframes on the column "Player" and "Full Name" or "Known As" and merge the column "Overall" from df_fifa to df_all

#df_all = pd.merge(df_all, df_fifa[['Full Name', 'Overall']], left_on='Player', right_on='Full Name', how='left')
#df_all = pd.merge(df_all, df_fifa[['Known As', 'Overall']], left_on='Player', right_on='Known As', how='left')




In [10]:
# fill the NaN values if the column "Full Name" is in the column "Player" then fill the column "Overall" with the value from the column "Overall" from df_fifa

for index, row in df_all.iterrows():
    if row['Player'] in df_fifa['Full Name'].values:
        df_all.loc[index, 'Overall'] = df_fifa[df_fifa['Full Name'] == row['Player']]['Overall'].values[0]
    elif row['Player'] in df_fifa['Known As'].values:
        df_all.loc[index, 'Overall'] = df_fifa[df_fifa['Known As'] == row['Player']]['Overall'].values[0]
    else:
        df_all.loc[index, 'Overall'] = np.nan



In [11]:
# check where the column "Overall" is NaN
df_all[df_all['Overall'].isna()].tail(50)

,Pos.,Player,Caps,Goals,Club,Country,team_id,Leauge_country,Overall
629,MF,Son Heung-min,104,35,Tottenham Hotspur,South Korea,NaN,NaN,NaN
630,MF,Paik Seung-ho,14,2,Jeonbuk Hyundai Motors,South Korea,NaN,NaN,NaN
631,FW,Cho Gue-sung,16,4,Jeonbuk Hyundai Motors,South Korea,NaN,NaN,NaN
632,MF,Lee Jae-sung,64,9,Mainz 05,South Korea,NaN,NaN,NaN
633,MF,Hwang Hee-chan,49,9,Wolverhampton Wanderers,South Korea,NaN,NaN,NaN
634,GK,Song Bum-keun,1,0,Jeonbuk Hyundai Motors,South Korea,NaN,NaN,NaN
635,MF,Son Jun-ho,15,0,Shandong Taishan,South Korea,NaN,NaN,NaN
637,DF,Kim Moon-hwan,22,0,Jeonbuk Hyundai Motors,South Korea,NaN,NaN,NaN
638,FW,Hwang Ui-jo,49,16,Olympiacos,South Korea,NaN,NaN,NaN
639,MF,Na Sang-ho,24,2,FC Seoul,South Korea,2766.0,South Korea,NaN


In [13]:
# Exportet the csv file and typed in the last players manually in the column "Overall"
# import the data file "all_list_with_overall.csv" 
df_all = pd.read_csv("all_list_with_overall.csv", delimiter=";")


df_all.head()


,Pos.,Player,Caps,Goals,Club,Country,team_id,Leauge_country,Overall
0,GK,Hernan Galindez,12,0,Aucas,Ecuador,1156.00.00,Equador,NaN
1,DF,Felix Torres,17,2,Santos Laguna,Ecuador,2285.00.00,mexico,NaN
2,DF,Piero Hincapie,21,1,Bayer Leverkusen,Ecuador,168.00.00,Germany,78.00.00
3,DF,Robert Arboleda,33,2,Sao Paulo,Ecuador,NaN,NaN,NaN
4,MF,Jose Cifuentes,11,0,Los Angeles FC,Ecuador,1616.00.00,USA,73.00.00


In [14]:
# print the sum of nan values in the column "Overall"
print(df_all['Overall'].isna().sum())



146


In [15]:
# replace the nan values with 0 
df_all['Overall'] = df_all['Overall'].fillna(0)
df_all.head()




,Pos.,Player,Caps,Goals,Club,Country,team_id,Leauge_country,Overall
0,GK,Hernan Galindez,12,0,Aucas,Ecuador,1156.00.00,Equador,0
1,DF,Felix Torres,17,2,Santos Laguna,Ecuador,2285.00.00,mexico,0
2,DF,Piero Hincapie,21,1,Bayer Leverkusen,Ecuador,168.00.00,Germany,78.00.00
3,DF,Robert Arboleda,33,2,Sao Paulo,Ecuador,NaN,NaN,0
4,MF,Jose Cifuentes,11,0,Los Angeles FC,Ecuador,1616.00.00,USA,73.00.00


In [16]:
# only keep the first 2 digits from the column "Overall"
df_all['Overall'] = df_all['Overall'].astype(str).str[:2]
df_all['Overall'] = df_all['Overall'].astype(int)
df_all.head()

,Pos.,Player,Caps,Goals,Club,Country,team_id,Leauge_country,Overall
0,GK,Hernan Galindez,12,0,Aucas,Ecuador,1156.00.00,Equador,0
1,DF,Felix Torres,17,2,Santos Laguna,Ecuador,2285.00.00,mexico,0
2,DF,Piero Hincapie,21,1,Bayer Leverkusen,Ecuador,168.00.00,Germany,78
3,DF,Robert Arboleda,33,2,Sao Paulo,Ecuador,NaN,NaN,0
4,MF,Jose Cifuentes,11,0,Los Angeles FC,Ecuador,1616.00.00,USA,73


In [17]:
# What is the average "overall" of the players
print(df_all[df_all['Overall'] != 0]['Overall'].mean())
# print the min and max value of the column "Overall"
print(df_all[df_all['Overall'] != 0]['Overall'].min())
print(df_all['Overall'].max())

# changing the value 0 to the average of the min value and the average value
df_all.loc[df_all['Overall'] == 0, 'Overall'] = int((df_all[df_all['Overall'] != 0]['Overall'].min() + df_all[df_all['Overall'] != 0]['Overall'].mean()) / 2)
df_all.head()






76.66959064327486
59
91


,Pos.,Player,Caps,Goals,Club,Country,team_id,Leauge_country,Overall
0,GK,Hernan Galindez,12,0,Aucas,Ecuador,1156.00.00,Equador,67
1,DF,Felix Torres,17,2,Santos Laguna,Ecuador,2285.00.00,mexico,67
2,DF,Piero Hincapie,21,1,Bayer Leverkusen,Ecuador,168.00.00,Germany,78
3,DF,Robert Arboleda,33,2,Sao Paulo,Ecuador,NaN,NaN,67
4,MF,Jose Cifuentes,11,0,Los Angeles FC,Ecuador,1616.00.00,USA,73


G = nx.from_pandas_edgelist(df, source = 'Player', target = 'Leauge_country', 
                        create_using = nx.DiGraph())


In [18]:
df_all

,Pos.,Player,Caps,Goals,Club,Country,team_id,Leauge_country,Overall
0,GK,Hernan Galindez,12,0,Aucas,Ecuador,1156.00.00,Equador,67
1,DF,Felix Torres,17,2,Santos Laguna,Ecuador,2285.00.00,mexico,67
2,DF,Piero Hincapie,21,1,Bayer Leverkusen,Ecuador,168.00.00,Germany,78
3,DF,Robert Arboleda,33,2,Sao Paulo,Ecuador,NaN,NaN,67
4,MF,Jose Cifuentes,11,0,Los Angeles FC,Ecuador,1616.00.00,USA,73
...,...,...,...,...,...,...,...,...,...
825,FW,Hirving Lozano,60,16,Napoli,Mexico,492.00.00,Italy,81
826,DF,Jesus Gallardo,78,1,Monterrey,Mexico,2282.00.00,mexico,75
827,MF,Luis Chavez,9,0,Pachuca,Mexico,2292.00.00,mexico,67
828,FW,Roberto Alvarado,31,4,Guadalajara,Mexico,NaN,NaN,67


In [19]:
# save the dataframe to a csv file named "FIFA23_Cleaned.csv"
df_all.to_csv("FIFA23_Cleaned.csv", index=False)

